# INF-395 Redes Neuronales Artificiales
## Tarea 1 - Redes Neuro

### Integrantes:
* Ignacio Valenzuelipico
* Rodrigo González Smith

# Introducción

# Desarrollo

Antes de proceder a realizar el trabajo importamos las librerías y paquetes que usaremos:

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

Using TensorFlow backend.
/home/ipresnya/aidPython/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## 1. Predicción de Entalpía de Atomización

###  1.a. Construir Dataframe

In [64]:
datos= pd.read_csv("roboBohr.csv")
datos.shape
datos.info()
datos.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16242 entries, 0 to 16241
Columns: 1278 entries, Unnamed: 0 to Eat
dtypes: float64(1276), int64(2)
memory usage: 158.4 MB


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id,Eat
count,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,...,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000
mean,8139.041805,115.715266,22.445723,20.474191,18.529573,17.169350,15.816888,15.133152,14.471534,13.960759,...,0.000134,0.000133,0.003879,0.000131,0.000129,0.002155,0.000127,0.001201,33107.484300,-11.178969
std,4698.182820,113.198503,8.659586,7.670481,6.485777,5.512560,4.179691,3.885091,3.503075,3.357136,...,0.002728,0.002705,0.043869,0.002676,0.002633,0.032755,0.002594,0.024472,23456.785147,3.659133
min,0.000000,36.858105,2.906146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-23.245373
25%,4068.250000,73.516695,17.969345,16.228071,15.165862,13.744092,13.653146,13.637784,12.759519,12.587359,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12298.250000,-13.475805
50%,8142.500000,73.516695,20.662511,18.631287,17.690729,16.020040,15.156646,13.848274,13.659233,13.652832,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27731.500000,-10.835211
75%,12207.750000,73.516695,21.132432,20.739496,18.712895,18.297501,17.639688,16.154918,15.499474,14.900585,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,55020.750000,-8.623903
max,16272.000000,388.023441,73.563510,66.269180,66.268891,66.268756,66.268196,66.264158,66.258487,66.258177,...,0.062225,0.061999,0.500000,0.061534,0.059760,0.500000,0.057834,0.500000,74980.000000,-0.789513


In [65]:
datos.drop(columns=['Unnamed: 0','pubchem_id'],axis=1,inplace=True)
total=len(datos)
df_train=datos[:int(0.6*total)]                       #60% de los datos
df_val=datos[int(0.6*total):int(0.85*total)]          #25% de los datos
df_test=datos[int(0.85*total)::]                      #15% restante

#### 1.a.1. Normalización de datos

In [66]:
scaler = StandardScaler().fit(df_train)
X_train_scaled = pd.DataFrame(scaler.transform(df_train),columns=df_train.columns)
X_val_scaled =  pd.DataFrame(scaler.transform(df_val),columns=df_val.columns)
X_test_scaled =  pd.DataFrame(scaler.transform(df_test),columns=df_test.columns)

In [67]:
y_train = df_train.pop('Eat').values.reshape(-1,1)
y_val = df_val.pop('Eat').values.reshape(-1,1)

In [68]:
X_train_scaled.drop(columns=['Eat'],axis=1,inplace=True)
X_val_scaled.drop(columns=['Eat'],axis=1,inplace=True)
X_test_scaled.drop(columns=['Eat'],axis=1,inplace=True)

### 1.b.  Red feedfoward

In [110]:
a = np.matrix(X_train_scaled)
a_val_scaled = np.matrix(X_val_scaled)

In [111]:
model = Sequential()
model.add(Dense(256, input_dim=X_train_scaled.shape[1], kernel_initializer='uniform',activation="sigmoid"))
model.add(Dense(1, kernel_initializer='uniform',activation="linear")) 
model.compile(optimizer=SGD(lr=0.01),loss='mean_squared_error')
history = model.fit(a, y_train, epochs=250, verbose=1, validation_data=(a_val_scaled, y_val))

Train on 9745 samples, validate on 4060 samples
Epoch 1/250
9745/9745 [==============================] - 4s 364us/step - loss: 1.3641 - val_loss: 0.5621
Epoch 2/250
9745/9745 [==============================] - 2s 254us/step - loss: 0.6090 - val_loss: 0.8801
Epoch 3/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.5136 - val_loss: 0.3744
Epoch 4/250
9745/9745 [==============================] - 2s 247us/step - loss: 0.4310 - val_loss: 0.3257
Epoch 5/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.3795 - val_loss: 0.4543
Epoch 6/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.3246 - val_loss: 0.2627
Epoch 7/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.2836 - val_loss: 0.2383
Epoch 8/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.2556 - val_loss: 0.2151
Epoch 9/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.2275 - val_loss: 0.1920
Epoch 1

Epoch 78/250
9745/9745 [==============================] - 2s 247us/step - loss: 0.0308 - val_loss: 0.0702
Epoch 79/250
9745/9745 [==============================] - 2s 250us/step - loss: 0.0298 - val_loss: 0.0615
Epoch 80/250
9745/9745 [==============================] - 3s 298us/step - loss: 0.0293 - val_loss: 0.0508
Epoch 81/250
9745/9745 [==============================] - 2s 247us/step - loss: 0.0297 - val_loss: 0.0484
Epoch 82/250
9745/9745 [==============================] - 3s 266us/step - loss: 0.0289 - val_loss: 0.0473
Epoch 83/250
9745/9745 [==============================] - 2s 246us/step - loss: 0.0291 - val_loss: 0.0736
Epoch 84/250
9745/9745 [==============================] - 3s 258us/step - loss: 0.0292 - val_loss: 0.0491
Epoch 85/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.0296 - val_loss: 0.0465
Epoch 86/250
9745/9745 [==============================] - 2s 244us/step - loss: 0.0283 - val_loss: 0.0463
Epoch 87/250
9745/9745 [======================

Epoch 155/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.0194 - val_loss: 0.0387
Epoch 156/250
9745/9745 [==============================] - 2s 252us/step - loss: 0.0197 - val_loss: 0.0393
Epoch 157/250
9745/9745 [==============================] - 3s 260us/step - loss: 0.0194 - val_loss: 0.0637
Epoch 158/250
9745/9745 [==============================] - 2s 243us/step - loss: 0.0201 - val_loss: 0.0451
Epoch 159/250
9745/9745 [==============================] - 2s 244us/step - loss: 0.0197 - val_loss: 0.0398
Epoch 160/250
9745/9745 [==============================] - 3s 291us/step - loss: 0.0192 - val_loss: 0.0422
Epoch 161/250
9745/9745 [==============================] - 3s 273us/step - loss: 0.0192 - val_loss: 0.0422
Epoch 162/250
9745/9745 [==============================] - 3s 291us/step - loss: 0.0198 - val_loss: 0.0401
Epoch 163/250
9745/9745 [==============================] - 3s 259us/step - loss: 0.0198 - val_loss: 0.0400
Epoch 164/250
9745/9745 [============

9745/9745 [==============================] - 2s 245us/step - loss: 0.0147 - val_loss: 0.0378
Epoch 232/250
9745/9745 [==============================] - 2s 247us/step - loss: 0.0152 - val_loss: 0.0356
Epoch 233/250
9745/9745 [==============================] - 2s 246us/step - loss: 0.0159 - val_loss: 0.0387
Epoch 234/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.0153 - val_loss: 0.0364
Epoch 235/250
9745/9745 [==============================] - 2s 246us/step - loss: 0.0154 - val_loss: 0.0412
Epoch 236/250
9745/9745 [==============================] - 2s 246us/step - loss: 0.0151 - val_loss: 0.0364
Epoch 237/250
9745/9745 [==============================] - 2s 246us/step - loss: 0.0151 - val_loss: 0.0447
Epoch 238/250
9745/9745 [==============================] - 2s 246us/step - loss: 0.0146 - val_loss: 0.0356
Epoch 239/250
9745/9745 [==============================] - 2s 245us/step - loss: 0.0146 - val_loss: 0.0361
Epoch 240/250
9745/9745 [==========================